<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [6]</a>'.</span>

# Ingest toy version of MNIST digit data from sklearn

### Determine run parameters

In [1]:
# ----------------- Parameters for interactive development --------------
P = {
    "pipeline.run_environment": "dev",
    "pipeline.data_lake_root": "/pipeline-outputs/data-lake",
    "run.retry_nr": "1",
}

In [2]:
# - During automated runs parameters will be injected in the below cell -

In [3]:
# Parameters
P = {
    "pipeline.data_lake_root": "/pipeline-outputs/data-lake",
    "pipeline.run_environment": "ci",
    "pipeline.pipeline_run_id": "a6c4c4ae-4fdd-40fb-8a52-6e3e3984e925",
    "pipeline.github.repository": "pynb-dag-runner/mnist-digits-demo-pipeline",
    "pipeline.github.workflow": "Run automated tests, pipeline and deploy results to static reporting site",
    "pipeline.github.runner_name": "GitHub Actions 2",
    "pipeline.github.run_id": "2668450777",
    "pipeline.github.actor": "matiasdahl",
    "pipeline.github.job": "run-tests-pipeline-and-persist-pipeline-outputs",
    "pipeline.github.base_ref": "",
    "pipeline.github.head_ref": "",
    "pipeline.github.sha": "147a1be9916173fd4d2cd011d8d51e3411215850",
    "pipeline.github.ref": "refs/heads/development",
    "pipeline.github.ref_type": "branch",
    "pipeline.github.ref_name": "development",
    "pipeline.github.event_name": "schedule",
    "task.notebook": "notebooks/ingest.py",
    "task.max_nr_retries": "15",
    "run.retry_nr": "4",
    "task.timeout_s": "10",
    "task.num_cpus": 1,
    "_opentelemetry_traceparent": "00-8ed032a1273b4d8d9ed85f02e5fe0471-dbf530efeb1c0b46-01",
}


In [4]:
# -----------------------------------------------------------------------

---

### Simulate different types of failures (for testing timeout and retry logic)

In [5]:
from pynb_dag_runner.tasks.task_opentelemetry_logging import PydarLogger

logger = PydarLogger(P)

2022-07-14 17:36:55,101	INFO worker.py:842 -- Connecting to existing Ray cluster at address: 172.17.0.2:6379


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [6]:
import time, random


def maybe_crash(retry_nr: int, run_environment: str):
    if retry_nr == 2 and run_environment == "ci":
        time.sleep(1e6)

    max_retry_nr: int = 3 if run_environment == "dev" else 10

    if retry_nr < max_retry_nr:
        if random.random() < 0.1:
            time.sleep(1e6)
        else:
            raise Exception("Simulated exception failure from ingestion step notebook!")


maybe_crash(
    retry_nr=int(P["run.retry_nr"]), run_environment=P["pipeline.run_environment"]
)

Exception: Simulated exception failure from ingestion step notebook!

### Notebook code

In [ ]:
from sklearn import datasets

#
from common.io import datalake_root, write_numpy

In [ ]:
digits = datasets.load_digits()

X = digits["data"]
y = digits["target"]

In [ ]:
logger.log_value("data_shape", list(X.shape))
logger.log_value("target_shape", list(y.shape))

X.shape, y.shape

In [ ]:
write_numpy(datalake_root(P) / "raw" / "digits.numpy", X)
write_numpy(datalake_root(P) / "raw" / "labels.numpy", y)